In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.simplefilter('ignore')

In [19]:
df = pd.read_csv(r"E:\Data Science by SRK\Machine_learning\Classification\Gene_data\cleaned_dataset_Gene.csv")

In [20]:
df

,Gene_One,Gene_Two,Cancer_Present
0,4.3,3.9,1
1,2.5,6.3,0
2,5.7,3.9,1
3,6.1,6.2,0
4,7.4,3.4,1
...,...,...,...
2133,5.0,6.5,1
2134,3.4,6.6,0
2135,2.7,6.5,0
2136,3.3,5.6,0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138 entries, 0 to 2137
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gene_One        2138 non-null   float64
 1   Gene_Two        2138 non-null   float64
 2   Cancer_Present  2138 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 50.2 KB


In [22]:
df.isnull().sum()

Gene_One          0
Gene_Two          0
Cancer_Present    0
dtype: int64

# X and y

In [23]:
X = df.drop(columns = ['Cancer_Present'])
y = df['Cancer_Present']

# Train_test_split

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= True)

# ADABOOST
**Applying Hyperparameter Tuning for identifying best parameters for adaboost**

In [25]:
from sklearn.model_selection import GridSearchCV

# modelling

from sklearn.ensemble import AdaBoostClassifier
estimator_ab = AdaBoostClassifier()

# parameters grid
param_grid_ab = {"n_estimators" : list(range(1,100))}

# grid search
grid_ab = GridSearchCV(estimator_ab, param_grid_ab, cv = 5, scoring = 'accuracy')
grid_ab.fit(X_train, y_train)

# best parameters for adaboost model
grid_ab.best_params_

{'n_estimators': 29}

# ADA Boost model important features

In [26]:
feats_ab = pd.DataFrame(data = grid_ab.best_estimator_.feature_importances_,
                       index = X.columns, 
                       columns = ['Importance'])
imp_features_list_ab = feats_ab[feats_ab['Importance']>0].index.to_list()
imp_features_list_ab

['Gene_One', 'Gene_Two']

# ADABoost Classifier

**with imortant features and best hyperparameters**

In [29]:
# important input variables
X_imp = X[imp_features_list_ab]

# train test split
X_train_ab, X_test_ab, y_train_ab, y_test_ab = train_test_split(X_imp, y, test_size = 0.2, random_state = True)

# modelling
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators= 29)
ada.fit(X_train_ab, y_train_ab)

# Evaluation on train data
ypred_train_ab = ada.predict(X_train_ab)

from sklearn.metrics import accuracy_score
print("Train_accuracy : ", accuracy_score(y_train_ab, ypred_train_ab))

# cross validation
from sklearn.model_selection import cross_val_score
print("cross_val_score : ", cross_val_score(ada, X_train_ab, y_train_ab, cv = 5).mean())

# Evaluation
ypred_test_ab = ada.predict(X_test_ab)
print("Test_accuracy : ", accuracy_score(y_test_ab, ypred_test_ab))


Train_accuracy :  0.9146198830409357
cross_val_score :  0.9128654970760234
Test_accuracy :  0.9065420560747663


In [28]:
%%timeit
ada.fit(X_train_ab, y_train_ab)


119 ms ± 2.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
